In [1]:
import pandas as pd
import spacy
from spacy.language import Language  # For custom pipeline components
from spacy_langdetect import LanguageDetector  # For language detection
from tqdm import tqdm  # For progress bars
import re
from collections import Counter

**HELPER CODE - CONSTANTS AND FUNCTIONS**

In [11]:
# File names
folderName = '../Data/'
dataFile1 = 'CAT_Tweets_20250513.json'
lexiconFile = 'CAT_Lexicon.csv'

In [4]:
# FUNCTIONALITY NEEDED TO DETECT THE LANGUAGE IN THE TWEETS
# Load spacy model for Catalan
nlp = spacy.load("ca_core_news_md")

# Add language detector functionality into the pipeline
# Custom language detector factory function
@Language.factory("language_detector") 
def create_language_detector(nlp, name):
    return LanguageDetector() # Create the detector component

# Add language detector to the spaCy pipeline
nlp.add_pipe("language_detector", last=True)

# Function to check if the text is in catalan
def is_catalan(text):
    doc = nlp(text)  # Process the text with the spaCy pipeline
    detect_language = doc._.language  # Access language detection results
    #print(detect_language)
    return int(detect_language['language'] == 'ca' and detect_language['score'] > 0.9)  # Check if detected language is catalan and with high confidence

**LOAD DATA**

In [7]:
# Load the lexicon file - contains keywords and type of borrowing
df_lex = pd.read_csv(folderName+lexiconFile, sep=';')[['word_catalan', 'type_borrowing']]

In [8]:
df_lex.head()

,word_catalan,type_borrowing
0,m'agrada,Calque
1,likejar,Partial adaptation
2,comentar,Calque
3,comentari,Calque
4,compartir,Calque


In [12]:
# Open data file
df_init = pd.read_json(folderName+dataFile1)

In [13]:
df_init.head()

,avatar,id,text,likes,replies,retweets,quotes,timestamp,searchQuery,url,images
0,https://pbs.twimg.com/profile_images/157008166...,1619677524967190528,"Un ruzi*, que he barrat tot d'una, vota pel me...",0,0,0,0,2023-01-29 12:43:00+00:00,repiulet,https://x.com/nom00762250/status/1619677524967...,NaN
1,https://pbs.twimg.com/profile_images/148807679...,1620168513376894976,podeu demanar la dimissió de Sigfrid Gras sens...,3,0,0,0,2023-01-30 21:14:00+00:00,retuitar,https://x.com/VerdaMongeta/status/162016851337...,NaN
2,https://pbs.twimg.com/profile_images/146488290...,1619742793383170048,"Perquè, retuitar? Perque fa falta",3,0,2,0,2023-01-29 17:02:00+00:00,retuitar,https://x.com/JosepaMontagut/status/1619742793...,NaN
3,https://pbs.twimg.com/profile_images/870318866...,1619666435776864256,"En contra de retuitar genocides, per molt sucó...",0,0,0,0,2023-01-29 11:59:00+00:00,retuitar,https://x.com/Sergi1001/status/161966643577686...,NaN
4,https://pbs.twimg.com/profile_images/115014509...,1619642801406509056,Retuit si tu també creus que tampoc s'ha de re...,0,0,3,0,2023-01-29 10:25:00+00:00,retuitar,https://x.com/vas_tard/status/1619642801406509056,NaN


In [ ]:
df = df_init[['id', 'searchQuery', 'text', 'timestamp']] # Keep ony relevant columns in a new DF

In [15]:
print(df.shape)
df.head()

(9086, 4)


,id,searchQuery,text,timestamp
0,1619677524967190528,repiulet,"Un ruzi*, que he barrat tot d'una, vota pel me...",2023-01-29 12:43:00+00:00
1,1620168513376894976,retuitar,podeu demanar la dimissió de Sigfrid Gras sens...,2023-01-30 21:14:00+00:00
2,1619742793383170048,retuitar,"Perquè, retuitar? Perque fa falta",2023-01-29 17:02:00+00:00
3,1619666435776864256,retuitar,"En contra de retuitar genocides, per molt sucó...",2023-01-29 11:59:00+00:00
4,1619642801406509056,retuitar,Retuit si tu també creus que tampoc s'ha de re...,2023-01-29 10:25:00+00:00


**PRE-PROCESSING THE DATA**

In [16]:
# Remove rows that have empty text column (NA)
df = df.dropna(axis=0, subset=['text'], ignore_index=True)

In [17]:
print(df.shape)
df.head()

(8923, 4)


,id,searchQuery,text,timestamp
0,1619677524967190528,repiulet,"Un ruzi*, que he barrat tot d'una, vota pel me...",2023-01-29 12:43:00+00:00
1,1620168513376894976,retuitar,podeu demanar la dimissió de Sigfrid Gras sens...,2023-01-30 21:14:00+00:00
2,1619742793383170048,retuitar,"Perquè, retuitar? Perque fa falta",2023-01-29 17:02:00+00:00
3,1619666435776864256,retuitar,"En contra de retuitar genocides, per molt sucó...",2023-01-29 11:59:00+00:00
4,1619642801406509056,retuitar,Retuit si tu també creus que tampoc s'ha de re...,2023-01-29 10:25:00+00:00


In [18]:
# Convert to lowercase both searchQuery and text
df['searchQuery'] = df['searchQuery'].apply(str.lower)
df['text'] = df['text'].apply(str.lower)

In [19]:
# Keep only rows that contain the searchQuery in the text
df_filt = df[df.apply(lambda row: row['searchQuery'] in row['text'], axis=1)]

In [54]:
#df[df.apply(lambda row: row['searchQuery'] not in row['text'], axis=1)]

In [20]:
print(df_filt.shape)
df_filt.head()

(2022, 4)


,id,searchQuery,text,timestamp
0,1619677524967190528,repiulet,"un ruzi*, que he barrat tot d'una, vota pel me...",2023-01-29 12:43:00+00:00
1,1620168513376894976,retuitar,podeu demanar la dimissió de sigfrid gras sens...,2023-01-30 21:14:00+00:00
2,1619742793383170048,retuitar,"perquè, retuitar? perque fa falta",2023-01-29 17:02:00+00:00
3,1619666435776864256,retuitar,"en contra de retuitar genocides, per molt sucó...",2023-01-29 11:59:00+00:00
4,1619642801406509056,retuitar,retuit si tu també creus que tampoc s'ha de re...,2023-01-29 10:25:00+00:00


In [21]:
# Incorporate type of borrowing into the data by searchQuery
df_keys = pd.merge(df_filt, df_lex, how="left", left_on="searchQuery", right_on="word_catalan")
print(df_keys.shape)
df_keys.head(10)

(2022, 6)


,id,searchQuery,text,timestamp,word_catalan,type_borrowing
0,1619677524967190528,repiulet,"un ruzi*, que he barrat tot d'una, vota pel me...",2023-01-29 12:43:00+00:00,repiulet,Calque
1,1620168513376894976,retuitar,podeu demanar la dimissió de sigfrid gras sens...,2023-01-30 21:14:00+00:00,retuitar,Full adaptation
2,1619742793383170048,retuitar,"perquè, retuitar? perque fa falta",2023-01-29 17:02:00+00:00,retuitar,Full adaptation
3,1619666435776864256,retuitar,"en contra de retuitar genocides, per molt sucó...",2023-01-29 11:59:00+00:00,retuitar,Full adaptation
4,1619642801406509056,retuitar,retuit si tu també creus que tampoc s'ha de re...,2023-01-29 10:25:00+00:00,retuitar,Full adaptation
5,1619495681428959232,retuitar,já ia retuitar kkaakakkaakak vou ser assim🫶🏻,2023-01-29 00:40:00+00:00,retuitar,Full adaptation
6,1619493816993718272,retuitar,"@kanen49 si us plau, deixa de retuitar aquests...",2023-01-29 00:33:00+00:00,retuitar,Full adaptation
7,1620194628803694592,share,i’m glad to share that i’ve obtained a new cer...,2023-01-30 22:57:00+00:00,NaN,NaN
8,1620132857305001984,share,aneu perdent llençols. al final tindreu conten...,2023-01-30 18:52:00+00:00,NaN,NaN
9,1620207698993360896,post,aventura/tps/post-apocalíptico/zombies,2023-01-30 23:49:00+00:00,NaN,NaN


In [22]:
# Drop the word_catalan column
df_keys.drop("word_catalan", axis=1, inplace=True)

In [23]:
# The NaN values in type_borrowing correspond to the English words, therefore, they are direct borrowings
# Add this value to the DF
df_keys['type_borrowing'].fillna("Direct borrowing", inplace=True)
df_keys.head(10)

/var/folders/n5/fr7ssm7s57z_7psd2b6ddp9r0000gp/T/ipykernel_32643/2797210978.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_keys['type_borrowing'].fillna("Direct borrowing", inplace=True)


,id,searchQuery,text,timestamp,type_borrowing
0,1619677524967190528,repiulet,"un ruzi*, que he barrat tot d'una, vota pel me...",2023-01-29 12:43:00+00:00,Calque
1,1620168513376894976,retuitar,podeu demanar la dimissió de sigfrid gras sens...,2023-01-30 21:14:00+00:00,Full adaptation
2,1619742793383170048,retuitar,"perquè, retuitar? perque fa falta",2023-01-29 17:02:00+00:00,Full adaptation
3,1619666435776864256,retuitar,"en contra de retuitar genocides, per molt sucó...",2023-01-29 11:59:00+00:00,Full adaptation
4,1619642801406509056,retuitar,retuit si tu també creus que tampoc s'ha de re...,2023-01-29 10:25:00+00:00,Full adaptation
5,1619495681428959232,retuitar,já ia retuitar kkaakakkaakak vou ser assim🫶🏻,2023-01-29 00:40:00+00:00,Full adaptation
6,1619493816993718272,retuitar,"@kanen49 si us plau, deixa de retuitar aquests...",2023-01-29 00:33:00+00:00,Full adaptation
7,1620194628803694592,share,i’m glad to share that i’ve obtained a new cer...,2023-01-30 22:57:00+00:00,Direct borrowing
8,1620132857305001984,share,aneu perdent llençols. al final tindreu conten...,2023-01-30 18:52:00+00:00,Direct borrowing
9,1620207698993360896,post,aventura/tps/post-apocalíptico/zombies,2023-01-30 23:49:00+00:00,Direct borrowing


In [24]:
# Check if there are links in the text (FOR TESTING)
df_keys[df_keys['text'].str.contains('\\.\\w')]

,id,searchQuery,text,timestamp,type_borrowing
7,1620194628803694592,share,i’m glad to share that i’ve obtained a new cer...,2023-01-30 22:57:00+00:00,Direct borrowing
12,1620200448316821504,post,➡️ quackity aparece en el post de @/machika_yt...,2023-01-30 23:21:00+00:00,Direct borrowing
14,1620195449733881856,post,👑jackson en el post de neomediaworks en ig\n\n...,2023-01-30 23:01:00+00:00,Direct borrowing
20,1620192105514815488,post,post del escorial de madrid\ninstagram.com/p/c...,2023-01-30 22:47:00+00:00,Direct borrowing
35,1620164159718703104,comentari,just avui he tingut una discussió amb un compa...,2023-01-30 20:56:00+00:00,Calque
...,...,...,...,...,...
1926,1619495891416616960,publicar,acaba de publicar una foto en parque nacional ...,2023-01-29 00:41:00+00:00,Semantic extension
1927,1619491969532829696,publicar,acaba de publicar una foto en parque nacional ...,2023-01-29 00:25:00+00:00,Semantic extension
1935,1620084464864096256,amic,"🤗 encara no formes part del programa ""amics de...",2023-01-30 15:40:00+00:00,Calque
1940,1620077816296660992,amic,si més no un programa d'una ràdio de cincinnat...,2023-01-30 15:13:00+00:00,Calque


In [25]:
df_keys['text'][1935]

'🤗 encara no formes part del programa "amics de la fundació?"\n\n💜 si vols col·laborar amb els nostres projectes, ara tens l\'oportunitat de donar-hi suport: amb una petita aportació podem fer grans coses!\n\n📍 fes-te amic aquí 👉 bit.ly/amicsdelafundacio'

In [61]:
'''t = df_keys['text'][1935]
print(t)

#re.findall(r'\n', t)
#re.findall(r'#\w*\s', t)
#re.findall(r'[#@]\w*\s', t)
t = re.sub(r'\n', ' ', t) # Remove newline chars
#print(t)
t = re.sub(r'(#|@)/?[\w\\]*\s', '', t) # Remove hashtags and mentions
re.sub(r'\s\w*\.?(\w*\/)+\w*', '', t)

#re.sub(r'\s{2,}', ' ', t) # Remove duplicated spaces'''

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/var/folders/n5/fr7ssm7s57z_7psd2b6ddp9r0000gp/T/ipykernel_28410/73039033.py:1: SyntaxWarning: invalid escape sequence '\w'
  '''t = df_keys['text'][1935]


"t = df_keys['text'][1935]\nprint(t)\n\n#re.findall(r'\n', t)\n#re.findall(r'#\\w*\\s', t)\n#re.findall(r'[#@]\\w*\\s', t)\nt = re.sub(r'\n', ' ', t) # Remove newline chars\n#print(t)\nt = re.sub(r'(#|@)/?[\\w\\]*\\s', '', t) # Remove hashtags and mentions\nre.sub(r'\\s\\w*\\.?(\\w*\\/)+\\w*', '', t)\n\n#re.sub(r'\\s{2,}', ' ', t) # Remove duplicated spaces"

In [26]:
# Clean text: remove newline chars, links, hashtags
def cleanText(t: str):
    t = re.sub(r'\n', ' ', t) # Remove newline chars
    t = re.sub(r'\s\w*\.?(\w*\/)+\w*', '', t) # Remove links
    t = re.sub(r'#[\w\\]*\s', '', t) # Remove hashtags
    t = re.sub(r'\s{2,}', ' ', t) # Remove duplicated spaces
    return t

In [27]:
cleanText(df_keys['text'][1935])

'🤗 encara no formes part del programa "amics de la fundació?" 💜 si vols col·laborar amb els nostres projectes, ara tens l\'oportunitat de donar-hi suport: amb una petita aportació podem fer grans coses! 📍 fes-te amic aquí 👉'

In [28]:
# Clean text: remove newline chars, links, hashtags
df_keys['cleanText'] = df_keys['text'].apply(cleanText)

In [29]:
print(df_keys.shape)
df_keys.head()

(2022, 6)


,id,searchQuery,text,timestamp,type_borrowing,cleanText
0,1619677524967190528,repiulet,"un ruzi*, que he barrat tot d'una, vota pel me...",2023-01-29 12:43:00+00:00,Calque,"un ruzi*, que he barrat tot d'una, vota pel me..."
1,1620168513376894976,retuitar,podeu demanar la dimissió de sigfrid gras sens...,2023-01-30 21:14:00+00:00,Full adaptation,podeu demanar la dimissió de sigfrid gras sens...
2,1619742793383170048,retuitar,"perquè, retuitar? perque fa falta",2023-01-29 17:02:00+00:00,Full adaptation,"perquè, retuitar? perque fa falta"
3,1619666435776864256,retuitar,"en contra de retuitar genocides, per molt sucó...",2023-01-29 11:59:00+00:00,Full adaptation,"en contra de retuitar genocides, per molt sucó..."
4,1619642801406509056,retuitar,retuit si tu també creus que tampoc s'ha de re...,2023-01-29 10:25:00+00:00,Full adaptation,retuit si tu també creus que tampoc s'ha de re...


In [30]:
# Language detection
# Add row to indicate if the tweet is written in Catalan or not (1 - CAT; 0 - NOT CAT)
df_keys['is_catalan'] = df_keys['text'].apply(is_catalan)

In [31]:
print(df_keys.shape)
df_keys.head()

(2022, 7)


,id,searchQuery,text,timestamp,type_borrowing,cleanText,is_catalan
0,1619677524967190528,repiulet,"un ruzi*, que he barrat tot d'una, vota pel me...",2023-01-29 12:43:00+00:00,Calque,"un ruzi*, que he barrat tot d'una, vota pel me...",1
1,1620168513376894976,retuitar,podeu demanar la dimissió de sigfrid gras sens...,2023-01-30 21:14:00+00:00,Full adaptation,podeu demanar la dimissió de sigfrid gras sens...,1
2,1619742793383170048,retuitar,"perquè, retuitar? perque fa falta",2023-01-29 17:02:00+00:00,Full adaptation,"perquè, retuitar? perque fa falta",1
3,1619666435776864256,retuitar,"en contra de retuitar genocides, per molt sucó...",2023-01-29 11:59:00+00:00,Full adaptation,"en contra de retuitar genocides, per molt sucó...",1
4,1619642801406509056,retuitar,retuit si tu també creus que tampoc s'ha de re...,2023-01-29 10:25:00+00:00,Full adaptation,retuit si tu també creus que tampoc s'ha de re...,1


In [32]:
df_keys[df_keys['is_catalan'] == 0].shape # Not Catalan

(575, 7)

In [33]:
df_keys[df_keys['is_catalan'] == 1].shape # Catalan

(1447, 7)

In [34]:
# Filter - keep only the rows that are in Catalan
df_cat = df_keys[df_keys['is_catalan'] == 1]
df_cat.drop('is_catalan', axis=1, inplace=True)

/var/folders/n5/fr7ssm7s57z_7psd2b6ddp9r0000gp/T/ipykernel_32643/3442522805.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat.drop('is_catalan', axis=1, inplace=True)


In [35]:
print(df_cat.shape)
df_cat.head()

(1447, 6)


,id,searchQuery,text,timestamp,type_borrowing,cleanText
0,1619677524967190528,repiulet,"un ruzi*, que he barrat tot d'una, vota pel me...",2023-01-29 12:43:00+00:00,Calque,"un ruzi*, que he barrat tot d'una, vota pel me..."
1,1620168513376894976,retuitar,podeu demanar la dimissió de sigfrid gras sens...,2023-01-30 21:14:00+00:00,Full adaptation,podeu demanar la dimissió de sigfrid gras sens...
2,1619742793383170048,retuitar,"perquè, retuitar? perque fa falta",2023-01-29 17:02:00+00:00,Full adaptation,"perquè, retuitar? perque fa falta"
3,1619666435776864256,retuitar,"en contra de retuitar genocides, per molt sucó...",2023-01-29 11:59:00+00:00,Full adaptation,"en contra de retuitar genocides, per molt sucó..."
4,1619642801406509056,retuitar,retuit si tu també creus que tampoc s'ha de re...,2023-01-29 10:25:00+00:00,Full adaptation,retuit si tu també creus que tampoc s'ha de re...


In [37]:
# Save clean dataset into csv
df_cat.to_csv(folderName+"cleanDataset.csv", sep=";")

In [39]:
# Tokenize
d = nlp(df_cat['cleanText'][4])
[t.text for t in d]

['retuit',
 'si',
 'tu',
 'també',
 'creus',
 'que',
 'tampoc',
 "s'",
 'ha',
 'de',
 'retuitar',
 'tot']

In [40]:
# Tokenize the text
df_cat['tokenized'] = df_cat['cleanText'].apply(lambda row: [tok.text.strip() for tok in nlp(row)]) #We use strip to remove any leading and trailing whitespaces

/var/folders/n5/fr7ssm7s57z_7psd2b6ddp9r0000gp/T/ipykernel_32643/1705272850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['tokenized'] = df_cat['cleanText'].apply(lambda row: [tok.text.strip() for tok in nlp(row)]) #We use strip to remove any leading and trailing whitespaces


In [41]:
df_cat.head()

,id,searchQuery,text,timestamp,type_borrowing,cleanText,tokenized
0,1619677524967190528,repiulet,"un ruzi*, que he barrat tot d'una, vota pel me...",2023-01-29 12:43:00+00:00,Calque,"un ruzi*, que he barrat tot d'una, vota pel me...","[un, ruzi, *, ,, que, he, barrat, tot, d', una..."
1,1620168513376894976,retuitar,podeu demanar la dimissió de sigfrid gras sens...,2023-01-30 21:14:00+00:00,Full adaptation,podeu demanar la dimissió de sigfrid gras sens...,"[podeu, demanar, la, dimissió, de, sigfrid, gr..."
2,1619742793383170048,retuitar,"perquè, retuitar? perque fa falta",2023-01-29 17:02:00+00:00,Full adaptation,"perquè, retuitar? perque fa falta","[perquè, ,, retuitar, ?, perque, fa, falta]"
3,1619666435776864256,retuitar,"en contra de retuitar genocides, per molt sucó...",2023-01-29 11:59:00+00:00,Full adaptation,"en contra de retuitar genocides, per molt sucó...","[en, contra, de, retuitar, genocides, ,, per, ..."
4,1619642801406509056,retuitar,retuit si tu també creus que tampoc s'ha de re...,2023-01-29 10:25:00+00:00,Full adaptation,retuit si tu també creus que tampoc s'ha de re...,"[retuit, si, tu, també, creus, que, tampoc, s'..."


In [42]:
# Search keyword in the text
row = 0
t = df_cat['tokenized'][row]
k = df_cat['searchQuery'][row]
print(t, k)

['un', 'ruzi', '*', ',', 'que', 'he', 'barrat', 'tot', "d'", 'una', ',', 'vota', 'p', 'el', 'meu', 'repiulet', '.', 'no', 'saben', 'ni', 'llegir', '.', '*', 'aquest', ',', 'nogensmenys', ',', 'també', 'era', 'partidari', 'de', 'les', 'dictadures', 'nord-coreana', ',', 'veneçolana', '...', 'i', 'ja', 'no', 'he', 'hi', 'volgut', 'veure', 'pas', 'més', '.'] repiulet


In [43]:
# Given a list of tokenized text, find the keyword and extract a context surrounding it
def extractContext(lst, kw, window = 5):
    try:
        idx = lst.index(kw)
        ini = idx - window if idx - window >= 0 else 0
        end = idx + window if idx + window <= len(lst) else len(lst)
        return ' '.join(lst[ini:end])
    except:
        return ''

In [44]:
df_cat['context'] = df_cat.apply(lambda row: extractContext(row['tokenized'], row['searchQuery'], 6), axis=1)

/var/folders/n5/fr7ssm7s57z_7psd2b6ddp9r0000gp/T/ipykernel_32643/1720226236.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['context'] = df_cat.apply(lambda row: extractContext(row['tokenized'], row['searchQuery'], 6), axis=1)


In [45]:
df_cat.head()

,id,searchQuery,text,timestamp,type_borrowing,cleanText,tokenized,context
0,1619677524967190528,repiulet,"un ruzi*, que he barrat tot d'una, vota pel me...",2023-01-29 12:43:00+00:00,Calque,"un ruzi*, que he barrat tot d'una, vota pel me...","[un, ruzi, *, ,, que, he, barrat, tot, d', una...","una , vota p el meu repiulet . no saben ni llegir"
1,1620168513376894976,retuitar,podeu demanar la dimissió de sigfrid gras sens...,2023-01-30 21:14:00+00:00,Full adaptation,podeu demanar la dimissió de sigfrid gras sens...,"[podeu, demanar, la, dimissió, de, sigfrid, gr...",la dimissió de sigfrid gras sense retuitar fei...
2,1619742793383170048,retuitar,"perquè, retuitar? perque fa falta",2023-01-29 17:02:00+00:00,Full adaptation,"perquè, retuitar? perque fa falta","[perquè, ,, retuitar, ?, perque, fa, falta]","perquè , retuitar ? perque fa falta"
3,1619666435776864256,retuitar,"en contra de retuitar genocides, per molt sucó...",2023-01-29 11:59:00+00:00,Full adaptation,"en contra de retuitar genocides, per molt sucó...","[en, contra, de, retuitar, genocides, ,, per, ...","en contra de retuitar genocides , per molt sucós"
4,1619642801406509056,retuitar,retuit si tu també creus que tampoc s'ha de re...,2023-01-29 10:25:00+00:00,Full adaptation,retuit si tu també creus que tampoc s'ha de re...,"[retuit, si, tu, també, creus, que, tampoc, s'...",creus que tampoc s' ha de retuitar tot


In [46]:
df_cat[df_cat['context'].str.contains('like')] # Construccions amb like: donar like, posar like

,id,searchQuery,text,timestamp,type_borrowing,cleanText,tokenized,context
40,1620152385334284288,comentari,soc un mindundi i no és representatiu de res p...,2023-01-30 20:10:00+00:00,Calque,soc un mindundi i no és representatiu de res p...,"[soc, un, mindundi, i, no, és, representatiu, ...",divisió per veure que el meu comentari té un l...
240,1619740595924373504,retuit,fem pressupostos després d' estudiar cada clie...,2023-01-29 16:53:00+00:00,Full adaptation,fem pressupostos després d' estudiar cada clie...,"[fem, pressupostos, després, d, ', estudiar, c...",""" . agrairia un like o retuit @_futbolero _ @a..."
320,1620166993092710400,tweet,ori me la pelen els teus albums com tornes a d...,2023-01-30 21:08:00+00:00,Direct borrowing,ori me la pelen els teus albums com tornes a d...,"[ori, me, la, pelen, els, teus, albums, com, t...",com tornes a donar like algun tweet insultan a...
424,1620076592105148416,publicació,sortegem dues entrades dobles 🎟 què cal fer? 😜...,2023-01-30 15:08:00+00:00,Semantic extension,sortegem dues entrades dobles 🎟 què cal fer? 😜...,"[sortegem, dues, entrades, dobles, 🎟, què, cal...",cultura_lleida 2️⃣ posa like a la publicació 3...
1033,1620174425835585536,like,"""ja no participis més""\n\nah però bé que us ag...",2023-01-30 21:37:00+00:00,Direct borrowing,"""ja no participis més"" ah però bé que us agrad...","["", ja, no, participis, més, "", ah, però, bé, ...",els meus tuits ehh ... mig like i faig una prè...
1036,1620173178738978816,like,"el policia nacional que es fa les cupaires, el...",2023-01-30 21:32:00+00:00,Direct borrowing,"el policia nacional que es fa les cupaires, el...","[el, policia, nacional, que, es, fa, les, cupa...",86 continuat . and i fucking like it .
1401,1619772707524182016,tweet,disculpeu. com que en això dels seguiments em ...,2023-01-29 19:01:00+00:00,Direct borrowing,disculpeu. com que en això dels seguiments em ...,"[disculpeu, ., com, que, en, això, d, els, seg...","poseu -me like en aquest mateix tweet , o a in..."


In [47]:
df_cat[df_cat['context'].str.contains('agrada')] 

,id,searchQuery,text,timestamp,type_borrowing,cleanText,tokenized,context
169,1620189215916179456,tweet,estic veient que la boya li agrada el tweet. i...,2023-01-30 22:36:00+00:00,Direct borrowing,estic veient que la boya li agrada el tweet. i...,"[estic, veient, que, la, boya, li, agrada, el,...",que la boya li agrada el tweet . i mentrestant...
264,1619697076832714752,clavar,no m’agrada clavar ideologia pel mig i menys e...,2023-01-29 14:00:00+00:00,Calque,no m’agrada clavar ideologia pel mig i menys e...,"[no, m’, agrada, clavar, ideologia, p, el, mig...",no m’ agrada clavar ideologia p el mig i
446,1620088426979790848,comentar,el barça no té director esportiu desde fa mass...,2023-01-30 15:55:00+00:00,Calque,el barça no té director esportiu desde fa mass...,"[el, barça, no, té, director, esportiu, desde,...",agradarien per cobrir les mancances . comentar...
636,1620175100246102016,seguidor,me n'he fet seguidor. m'ha agradat molt com l'...,2023-01-30 21:40:00+00:00,Calque,me n'he fet seguidor. m'ha agradat molt com l'...,"[me, n', he, fet, seguidor, ., m', ha, agradat...",me n' he fet seguidor . m' ha agradat molt
681,1620179342453248000,seguir,molt bona sèrie m'agrada i espero seguir cada ...,2023-01-30 21:57:00+00:00,Calque,molt bona sèrie m'agrada i espero seguir cada ...,"[molt, bona, sèrie, m', agrada, i, espero, seg...",bona sèrie m' agrada i espero seguir cada dill...
735,1620183709692608512,vist,us trobarem a faltar demà al matí... 😅 perquè ...,2023-01-30 22:14:00+00:00,Calque,us trobarem a faltar demà al matí... 😅 perquè ...,"[us, trobarem, a, faltar, demà, a, l, matí, .....",ens agrada dormir i us hauríem vist un cop més...
1079,1620120457830862848,resposta,una resposta molt desagradable.,2023-01-30 18:03:00+00:00,Calque,una resposta molt desagradable.,"[una, resposta, molt, desagradable, .]",una resposta molt desagradable .
1213,1619764663406968832,comentar,"25 anys! del ""crystal ball"" del geni. edicions...",2023-01-29 18:29:00+00:00,Calque,"25 anys! del ""crystal ball"" del geni. edicions...","[25, anys, !, d, el, "", crystal, ball, "", d, e...",", i també moltes coses per comentar . no va ag..."
1336,1619836182241902592,tweet,li dono al m'agrada perquè és l'única manera d...,2023-01-29 23:13:00+00:00,Direct borrowing,li dono al m'agrada perquè és l'única manera d...,"[li, dono, a, l, m', agrada, perquè, és, l', ú...",però m' agradaria més que aquest tweet no s' h...
1400,1619775733156704256,tweet,respond al rang d'edat: m'agrada tweet de cuin...,2023-01-29 19:13:00+00:00,Direct borrowing,respond al rang d'edat: m'agrada tweet de cuin...,"[respond, a, l, rang, d', edat, :, m', agrada,...",rang d' edat : m' agrada tweet de cuina nascut...


In [48]:
df_cat[df_cat['searchQuery'] == "m'agrada"]

,id,searchQuery,text,timestamp,type_borrowing,cleanText,tokenized,context
360,1620210027742269440,m'agrada,"sí, m'agrada molt com està.",2023-01-30 23:59:00+00:00,Calque,"sí, m'agrada molt com està.","[sí, ,, m', agrada, molt, com, està, .]",
361,1620202678088511488,m'agrada,"m'agrada....., simple independència ja!!!!!",2023-01-30 23:29:00+00:00,Calque,"m'agrada....., simple independència ja!!!!!","[m', agrada, ....., ,, simple, independència, ...",
362,1620202294963994624,m'agrada,estic molt feliç perque la meva vida és molt d...,2023-01-30 23:28:00+00:00,Calque,estic molt feliç perque la meva vida és molt d...,"[estic, molt, feliç, perque, la, meva, vida, é...",
363,1620201177001623552,m'agrada,"m'agrada, totalment d'acord amb joel....., sim...",2023-01-30 23:23:00+00:00,Calque,"m'agrada, totalment d'acord amb joel....., sim...","[m', agrada, ,, totalment, d', acord, amb, joe...",
364,1620201168445255680,m'agrada,"i m'agrada tant que igual m'ho tatuo, perque e...",2023-01-30 23:23:00+00:00,Calque,"i m'agrada tant que igual m'ho tatuo, perque e...","[i, m', agrada, tant, que, igual, m', ho, tatu...",
...,...,...,...,...,...,...,...,...
2014,1619870735878717440,m'agrada,"això m'agrada, quan no es sap que dir d'insulte",2023-01-30 01:30:00+00:00,Calque,"això m'agrada, quan no es sap que dir d'insulte","[això, m', agrada, ,, quan, no, es, sap, que, ...",
2015,1619855734556205056,m'agrada,"és curiós, perquè després aquesta gent (ell i ...",2023-01-30 00:31:00+00:00,Calque,"és curiós, perquè després aquesta gent (ell i ...","[és, curiós, ,, perquè, després, aquesta, gent...",
2017,1654790743498584064,m'agrada,wakanda forever\n6'5/10\nla peli és llarguíssi...,2023-05-06 10:10:00+00:00,Calque,wakanda forever 6'5/10 la peli és llarguíssima...,"[wakanda, forever, 6'5/10, la, peli, és, llarg...",
2018,1659251364411604992,m'agrada,ant-man y la avispa:quantumanía\n7/10\nla pelí...,2023-05-18 17:35:00+00:00,Calque,ant-man y la avispa:quantumanía la película no...,"[ant-man, y, la, avispa, :, quantumanía, la, p...",


**DATA ANALYSIS**

In [49]:
# See if there is any trend in regards to types of borrowings in the data
Counter(df_cat['type_borrowing'])

Counter({'Calque': 1083,
         'Semantic extension': 156,
         'Direct borrowing': 121,
         'Full adaptation': 87})

In [50]:
# Extract small window around searchQuery
# For each keyword, go through all tweets and extract a context
searchQueries = sorted(['like', 'comment', 'share', 'story', 'post', 'profile', 'follow', 'unfollow', 'reply', 'direct message', 'message', 'follower', 'tag', 'block', 'friend', 'tweet', 'retweet', 
                 'pin', 'seen', 'ban', 'mute', 'repost', 'subscribe', "m'agrada", 'likejar', 'comentar', 'comentari', 'compartir', 'història', 'publicació', 'publicar', 'perfil', 'seguir', 
                 'deixar de seguir', 'respondre', 'resposta', 'missatge directe', 'missatge', 'seguidor', 'etiqueta', 'etiquetar', 'bloquejar', 'blocar', 'amic', 'piular', 'tuitar', 'tuitejar', 
                 'piulada', 'tuit', 'piulet', 'repiular', 'retuitar', 'retuitejar', 'repiulada', 'retuit', 'repiulet', 'clavar', 'vist', 'prohibir', 'silenciar', 'republicar'])

In [56]:
df_cat['text'][1]

'podeu demanar la dimissió de sigfrid gras sense retuitar feixistes catalans és totalment compatible'

In [53]:
df_cat['id'][1]

np.int64(1620168513376894976)

In [55]:
df_cat['tokenized'][1][:10]

['podeu',
 'demanar',
 'la',
 'dimissió',
 'de',
 'sigfrid',
 'gras',
 'sense',
 'retuitar',
 'feixistes']

In [57]:
contexts = []

'''row = df_cat['tokenized'][3159]
keywords = list(set(searchQueries))
index = 1619772707524182016
window = 5'''

# For each row, go through the whole keyword list to find contexts
# If found, a tuple is inserted into the contexts list with the info: id of the tweet, keyword, text surrounding the keyword (given the window size)
def getContextForKeywords(tokenText, keywords, index, window): # CHANGE INDEX FOR ID? OR TRY TO SEE HOW TO EXTRACT INDEX FROM A DF INSIDE APPLY

    for k in keywords:
        result = extractContext(tokenText, k, window) # Returns a string with the context of the keyword (given a window)
        if result != '':
            dat = (index, k, result)
            contexts.append(dat)
    


In [58]:
contexts

[]

In [59]:
contexts = []
df_cat.apply(lambda row: getContextForKeywords(row['tokenized'], searchQueries, row['id'], 5), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
2014    None
2015    None
2017    None
2018    None
2020    None
Length: 1447, dtype: object

In [60]:
contexts[:5]

[(1619677524967190528, 'repiulet', ', vota p el meu repiulet . no saben ni'),
 (1620168513376894976,
  'retuitar',
  'dimissió de sigfrid gras sense retuitar feixistes catalans és totalment'),
 (1619742793383170048, 'retuitar', 'perquè , retuitar ? perque fa falta'),
 (1619666435776864256,
  'retuitar',
  'en contra de retuitar genocides , per molt'),
 (1619642801406509056, 'retuit', 'retuit si tu també creus')]

In [61]:
df_contexts = pd.DataFrame(contexts, columns=['id', 'keyword', 'context'])

In [62]:
print(df_contexts.shape)
df_contexts.head()

(1366, 3)


,id,keyword,context
0,1619677524967190528,repiulet,", vota p el meu repiulet . no saben ni"
1,1620168513376894976,retuitar,dimissió de sigfrid gras sense retuitar feixis...
2,1619742793383170048,retuitar,"perquè , retuitar ? perque fa falta"
3,1619666435776864256,retuitar,"en contra de retuitar genocides , per molt"
4,1619642801406509056,retuit,retuit si tu també creus


In [63]:
Counter(df_contexts['keyword'])

Counter({'seguir': 182,
         'missatge': 147,
         'publicar': 136,
         'compartir': 94,
         'amic': 94,
         'tuit': 72,
         'tweet': 69,
         'resposta': 68,
         'comentar': 62,
         'respondre': 55,
         'vist': 51,
         'prohibir': 40,
         'piular': 23,
         'piulada': 21,
         'publicació': 20,
         'comentari': 19,
         'història': 19,
         'silenciar': 19,
         'blocar': 18,
         'bloquejar': 16,
         'etiqueta': 13,
         'seguidor': 12,
         'retuit': 11,
         'block': 11,
         'clavar': 11,
         'perfil': 10,
         'post': 9,
         'like': 8,
         'ban': 8,
         'pin': 7,
         'tuitejar': 7,
         'retuitar': 6,
         'etiquetar': 6,
         'story': 4,
         'comment': 3,
         'friend': 2,
         'follow': 2,
         'piulet': 2,
         'tag': 2,
         'repiulet': 1,
         'share': 1,
         'retuitejar': 1,
         'mute': 1,
